In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [7]:
# model parameters
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle'
batch_size = 32
dim = 256
epochs = 50
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4
weight = 0.05
BatchNorm,useMax,interleave = True,True,False
logLoss = False
threshold = 0

In [ ]:
train_dl,val_dl = get_data(data,batch_size)

In [8]:
model = GNN_MataLayer(head,head_mol2,head_atom,head_edge,\
                      dim,layer1,layer2,factor,**data_dict[data],\
                      BatchNorm=BatchNorm,useMax=useMax,interleave=interleave).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

model,train_loss_perType,val_loss_perType,bestWeight = train_type_earlyStop(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                                                scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)

epoch:0, train_loss: +1.756, val_loss: -0.316, 
train_vector: +1.15|+0.63|+0.23|-0.28|-0.35|+0.29|-0.24|-0.70, 
val_vector  : +0.39|+0.43|-0.21|-0.26|-0.91|-0.14|-0.73|-1.10

epoch:1, train_loss: +0.840, val_loss: -0.630, 
train_vector: +0.65|+0.15|-0.31|-0.90|-1.04|-0.23|-0.88|-1.18, 
val_vector  : +0.50|-0.03|-0.54|-0.98|-1.21|-0.38|-1.04|-1.36



KeyboardInterrupt: 